# Bases da Datos

Prompt:  

este es un prompt de ejamplo para usar un llm que nos ayude:

"Sugiere una configuración de Python ligera pero potente que use una base de datos basada en archivos para poder practicar con bases de datos. Quiero que el código pueda ejecutarse en Google Colab."

1. que opinas del prompt
2. que responderias al prompt 

1. ¿Qué opino del prompt?

    El prompt está bastante bien planteado: es claro, tiene un objetivo (practicar con bases de datos), especifica el entorno (Google Colab) y da una restricción técnica razonable (base de datos basada en archivos, ligera pero potente).

Algunas sugerencias menores para hacerlo aún más efectivo:

- Aclarar si se quiere usar SQL directamente o también experimentar con ORM.
- Mencionar si se desea trabajar en modo interactivo (Jupyter) o con scripts.

Versión mejorada del prompt (opcional):

```textpain
“Sugiere una configuración ligera en Python para practicar con bases de datos en Google Colab, usando una base de datos basada en archivos (como SQLite). Quiero que el entorno sea fácil de usar, preferentemente con SQL y también opcionalmente con SQLAlchemy (ORM).”
```

2. ¿Qué respondería al prompt?

    Aquí va una posible respuesta útil del LLM:

Para practicar bases de datos en Google Colab con una configuración ligera pero potente, te recomiendo usar SQLite junto con las librerías estándar de Python y, opcionalmente, SQLAlchemy para trabajar con ORM.

- ✅ Configuración recomendada:

```python
# Instalar SQLAlchemy si querés usar ORM
!pip install sqlalchemy
```

- 🔧 Librerías a usar:
  - sqlite3: viene con Python, permite ejecutar consultas SQL directamente.
  - SQLAlchemy: opcional, permite trabajar con bases de datos usando clases y objetos de Python (ORM).
  - 

- 🗃️ Base de datos basada en archivo:

    SQLite crea un archivo .db que contiene toda la base de datos. Es ideal para Colab porque:

  - No necesita instalación de servidor.
  - Se puede guardar en el sistema de archivos de Colab o descargarlo.

## 🧪 Ejemplo básico con sqlite3:

In [ ]:
import sqlite3

In [ ]:
# Crear conexión y cursor
conn = sqlite3.connect("mi_base_de_datos.db")
cursor = conn.cursor()

In [ ]:
# Crear tabla
cursor.execute("CREATE TABLE IF NOT EXISTS alumnos (id INTEGER PRIMARY KEY, nombre TEXT)")

In [ ]:
# Insertar datos
cursor.execute("INSERT INTO alumnos (nombre) VALUES (?)", ("Ana",))
conn.commit()

In [ ]:
# Leer datos
cursor.execute("SELECT * FROM alumnos")
print(cursor.fetchall())

Cargar varios registros desde una lista de tuplas

In [ ]:
# Lista de alumnos
alumnos = [("Luis",), ("María",), ("Pedro",), ("Lucía",)]

# Insertar múltiples filas con executemany
cursor.executemany("INSERT INTO alumnos (nombre) VALUES (?)", alumnos)
conn.commit()

In [ ]:
# Cerrar cpneccion a la base

conn.close()

- 🚀 Consejo extra:

Podés usar librerías como pandas para visualizar resultados de consultas en forma de tabla:

In [ ]:
import pandas as pd

conn = sqlite3.connect("mi_base_de_datos.db")
df = pd.read_sql_query("SELECT * FROM alumnos", conn)
df.head()


## Ejemplo del video del curso

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [ ]:
# Crea el "engine" para conectarse a una base de datos SQLite llamada
# echo=True: hace que SQLAlchemy muestre en consola todas las consultas SQL
engine = create_engine('sqlite:///ecommerce.db', echo=True) 

In [ ]:

with engine.connect() as connection:
    result = connection.execute(text("SELECT 1"))
    # Fetch and print the result
    print(result.scalar()) # This should print 1 if the connection is successful

### Otro ejemplo con mas datos y una base de datos en memoria

Para crear una base de datos en memoria (es decir, que no se guarda en disco y se borra al cerrar la conexión), el cambio clave está en la URL de conexión.

🧠 Cambios según la librería

- ✅ 1. Con sqlite3 (base en memoria)

    ```python
    import sqlite3

    conn = sqlite3.connect(":memory:")
    ```

":memory:" le indica a SQLite que cree una base temporal en la RAM.

Se borra cuando cerrás la conexión, Ideal para pruebas rápidas.

- ✅ 2. Con SQLAlchemy

    ```python
    from sqlalchemy import create_engine

    engine = create_engine("sqlite:///:memory:", echo=True)
    ```

"sqlite:///:memory:" hace lo mismo: base en RAM, no en archivo.

Recordá que cada conexión nueva crea una base distinta, así que si vas a usar engine.connect() varias veces, deberías usar la misma conexión o manejar un poolclass.

- ⚠️ Importante con SQLAlchemy

    Si vas a usar la base de datos en memoria con varios accesos, usá esto para mantener la misma conexión abierta:

    ```python
    from sqlalchemy.pool import StaticPool
    engine = create_engine(
        "sqlite:///:memory:",
        connect_args={"check_same_thread": False},
        poolclass=StaticPool,
        echo=True
    )
    ```
  

In [ ]:
from sqlalchemy import create_engine, text

In [ ]:
# Crear conexión a una base de datos SQLite en memoria ram
engine = create_engine("sqlite:///:memory:", echo=True)

# Crear tabla e insertar múltiples filas
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS alumnos"))
    conn.execute(text("CREATE TABLE alumnos (id INTEGER PRIMARY KEY, nombre TEXT)"))
    conn.execute(text("INSERT INTO alumnos (nombre) VALUES ('Ana'), ('Luis'), ('María')"))
    conn.commit()

In [ ]:
# Ejecutar consulta que devuelve varias filas
with engine.connect() as conn:
    result = conn.execute(text("SELECT nombre FROM alumnos"))
    
    # scalar() solo toma el primer valor de la primera fila
    print("Usando .scalar():", result.scalar())  # → Ana

⚠️ Importante

> Después de usar .scalar(), el cursor ya ha leído la primera fila, por lo que si luego querés recorrer el resto, tendrías que reejecutar la consulta o usar otro método como .fetchall() desde el principio.

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT nombre FROM alumnos"))
    print("Usando .fetchoone():", result.fetchone())  # → Ana

| Método        | Devuelve                                      | Uso típico                        |
| ------------- | --------------------------------------------- | --------------------------------- |
| `scalar()`    | Un solo valor (primera columna, primera fila) | Consultas tipo `SELECT COUNT(*)`  |
| `fetchone()`  | Una sola fila                                 | Cuando esperás un único resultado |
| `fetchall()`  | Lista de todas las filas                      | Para procesar todo el resultado   |
| `fetchmany()` | Lista parcial de filas                        | Si querés paginar o limitar       |
| Iteración     | Una fila por ciclo                            | Para recorrer grandes resultados  |


# Generacion de un Esquema

## Diseñe un esquema de base de datos para una aplicación de comercio electrónico con tablas para usuarios, productos, pedidos y order_items.

Claro, acá tenés un diseño básico pero funcional de un esquema de base de datos para una aplicación de comercio electrónico. Incluye las tablas principales: usuarios, productos, pedidos y order_items (ítems del pedido).

### 📦 Tablas y relaciones

1. usuarios
  - id: ID único del usuario (PK)
  - nombre: nombre del usuario
  - email: correo electrónico (único)
  - direccion: dirección de envío
  - fecha_creacion: fecha en que se registró

2. productos
  - id: ID del producto (PK)
  - nombre: nombre del producto
  - descripcion: texto opcional
  - precio: precio unitario
  - stock: cantidad disponible

3. pedidos
  - id: ID del pedido (PK)
  - usuario_id: FK → usuarios.id
   - fecha: fecha del pedido
  - estado: por ejemplo "pendiente", "enviado", "cancelado"

4. order_items
  - id: ID del ítem (PK)
  - pedido_id: FK → pedidos.id
  - producto_id: FK → productos.id
  - cantidad: cuántos productos de ese tipo se pidieron
  - precio_unitario: precio del producto al momento del pedido (puede cambiar luego en productos)

🔗 Relaciones

- Un usuario puede tener muchos pedidos
- Un pedido puede tener muchos ítems
- Un ítem pertenece a un solo producto

```sql
CREATE TABLE usuarios (
    id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    direccion TEXT,
    fecha_creacion TEXT
);

CREATE TABLE productos (
    id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    descripcion TEXT,
    precio REAL NOT NULL,
    stock INTEGER NOT NULL
);

CREATE TABLE pedidos (
    id INTEGER PRIMARY KEY,
    usuario_id INTEGER NOT NULL,
    fecha TEXT NOT NULL,
    estado TEXT,
    FOREIGN KEY (usuario_id) REFERENCES usuarios(id)
);

CREATE TABLE order_items (
    id INTEGER PRIMARY KEY,
    pedido_id INTEGER NOT NULL,
    producto_id INTEGER NOT NULL,
    cantidad INTEGER NOT NULL,
    precio_unitario REAL NOT NULL,
    FOREIGN KEY (pedido_id) REFERENCES pedidos(id),
    FOREIGN KEY (producto_id) REFERENCES productos(id)
);
```

📝 Notas útiles para sqlite3

SQLite sí soporta claves foráneas, pero tenés que activarlas si querés que se cumplan:

```python
cursor.execute("PRAGMA foreign_keys = ON;")
```

El tipo TEXT y REAL funcionan bien para strings y números con decimales, respectivamente.

Las fechas se guardan como texto (TEXT) en formato ISO (YYYY-MM-DD) por lo general.



### Diferencia entre motores sql

- 🟡 Diferencias para MySQL

- Tipo INTEGER PRIMARY KEY → AUTO_INCREMENT

    ```sql
    id INTEGER PRIMARY KEY AUTO_INCREMENT
    ```
    
- Tipo TEXT para fechas → usar DATETIME o TIMESTAMP

    ```sql
    fecha_creacion DATETIME
    fecha DATETIME
    ```

- Restricción de claves foráneas requiere motor InnoDB

Agregá esto al final si querés asegurarte:

```sql
ENGINE=InnoDB;
```

Nombres de tablas y columnas a veces necesitan cuidado si son reservados (como order)

En tu caso order_items está bien, pero order sería conflictivo.

- 🧩 Ejemplo (MySQL, modificada solo una tabla):

    ```sql
    CREATE TABLE usuarios (
        id INTEGER PRIMARY KEY AUTO_INCREMENT,
        nombre TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        direccion TEXT,
        fecha_creacion DATETIME
    ) ENGINE=InnoDB;
    ```

- 🔵 Diferencias para PostgreSQL

- Tipo INTEGER PRIMARY KEY → SERIAL PRIMARY KEY

    ```sql
    id SERIAL PRIMARY KEY

- Tipo de fechas: usá TIMESTAMP o DATE

    ```sql
    fecha_creacion TIMESTAMP
    fecha TIMESTAMP
    ```

No hace falta declarar motor (ENGINE), PostgreSQL no lo usa.

- Tipos TEXT están OK, pero podrías preferir VARCHAR(n) si querés límites.

- 🧩 Ejemplo (PostgreSQL, tabla modificada):

    ```sql
    CREATE TABLE usuarios (
        id SERIAL PRIMARY KEY,
        nombre TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        direccion TEXT,
        fecha_creacion TIMESTAMP
    );
    ```

✅ Resumen de cambios clave

| Elemento                       | SQLite                | MySQL                                | PostgreSQL           |
| ------------------------------ | --------------------- | ------------------------------------ | -------------------- |
| Clave primaria autoincremental | `INTEGER PRIMARY KEY` | `INTEGER PRIMARY KEY AUTO_INCREMENT` | `SERIAL PRIMARY KEY` |
| Fecha/hora                     | `TEXT`                | `DATETIME` o `TIMESTAMP`             | `TIMESTAMP`          |
| Motor de tabla                 | No aplica             | `ENGINE=InnoDB`                      | No aplica            |


# Usando SQLalchemy como ORM

## 🧱 ¿Qué es un ORM?

ORM significa Object-Relational Mapping (Mapeo Objeto-Relacional).
Es una técnica que permite trabajar con bases de datos relacionales (como SQLite, PostgreSQL, MySQL) usando clases y objetos en lugar de escribir consultas SQL manuales.

- Con un ORM:
  - Cada tabla de la base se representa como una clase.
  - Cada fila es una instancia de esa clase.
  - Cada columna es un atributo de la clase.

🧑‍💻 Ejemplo simple

En vez de hacer esto en SQL:

```sql
SELECT * FROM usuarios WHERE id = 1;
```

Con un ORM como SQLAlchemy podés hacer:

```python
usuario = session.query(Usuario).get(1)
```

- 🔧 ¿Para qué sirve?
  - Para escribir menos SQL manual.
  - Para trabajar con la base usando Python puro.
  - Para que el código sea más portable y mantenible.



| Aspecto                          | ✅ Ventajas del ORM                                           | ⚠️ Desventajas del ORM                                                 |
| -------------------------------- | ------------------------------------------------------------ | ---------------------------------------------------------------------- |
| **Productividad**                | Abstrae SQL, menos código repetido                           | Puede ocultar detalles importantes del SQL                             |
| **Mantenimiento**                | Cambios en la base se reflejan en clases                     | Refactorizar mal puede causar errores silenciosos                      |
| **Legibilidad**                  | Más legible para quienes conocen Python                      | Menos legible para quienes prefieren SQL                               |
| **Compatibilidad entre motores** | El mismo código puede funcionar en SQLite, MySQL, PostgreSQL | Algunos comportamientos pueden no ser idénticos                        |
| **Consultas complejas**          | Permite componer consultas con objetos                       | Consultas muy complejas pueden volverse verbosas                       |
| **Seguridad (inyección SQL)**    | Menor riesgo si se usa bien                                  | Menor riesgo, pero aún puede haber errores si se mezcla con SQL manual |
| **Rendimiento**                  | Adecuado para la mayoría de los casos                        | Puede ser menos eficiente que SQL optimizado                           |
| **Aprendizaje**                  | Más amigable si ya conocés Python                            | Requiere entender el mapeo ORM                                         |
| **Depuración**                   | Genera SQL automáticamente (puede verse con `echo=True`)     | A veces cuesta rastrear el SQL generado                                |
| **Relaciones entre tablas**      | Muy fácil de navegar con `relationship()`                    | Puede generar muchas consultas si no se usa bien (`n+1 queries`)       |


- 🧠 Conclusión

Usar un ORM como SQLAlchemy:

Es ideal para prototipos, apps pequeñas a medianas, y cuando priorizás productividad y mantenimiento.

Puede ser menos eficiente que SQL puro en proyectos con grandes volúmenes de datos o consultas muy especializadas.

! En SQLAlchemy (y en general en cualquier ORM), podés definir métodos personalizados dentro de cada clase del modelo. Esto es muy útil para:

- Encapsular lógica relacionada con esa entidad.
- Agregar validaciones.
- Realizar acciones relacionadas con sus atributos.

🧑‍💻 Ejemplo: agregar un método en una clase ORM

```python
class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nombre = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)

    def saludar(self):
        return f"Hola, soy {self.nombre}!"
```

Y podés usarlo así:

```python
usuario = session.query(Usuario).get(1)
print(usuario.saludar())  # → Hola, soy Juan!
```
📌 Recomendaciones

Los métodos no deben hacer commits directamente, salvo que tengas un caso muy claro.

Podés crear también métodos de clase (@classmethod) o estáticos (@staticmethod) si necesitás lógica más general.

También podés crear propiedades calculadas con @property.

🧱 Esquema completo en SQLAlchemy (usando SQLite)

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Text, Float, ForeignKey, DateTime
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from datetime import datetime

In [2]:
Base = declarative_base()

class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nombre = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)
    direccion = Column(Text)
    fecha_creacion = Column(DateTime, default=datetime.utcnow)

    pedidos = relationship('Pedido', back_populates='usuario')


class Producto(Base):
    __tablename__ = 'productos'
    id = Column(Integer, primary_key=True)
    nombre = Column(String, nullable=False)
    descripcion = Column(Text)
    precio = Column(Float, nullable=False)
    stock = Column(Integer, nullable=False)

    items = relationship('OrderItem', back_populates='producto')


class Pedido(Base):
    __tablename__ = 'pedidos'
    id = Column(Integer, primary_key=True)
    usuario_id = Column(Integer, ForeignKey('usuarios.id'), nullable=False)
    fecha = Column(DateTime, default=datetime.utcnow)
    estado = Column(String)

    usuario = relationship('Usuario', back_populates='pedidos')
    items = relationship('OrderItem', back_populates='pedido')

    def total(self):
        return sum(item.cantidad * item.precio_unitario for item in self.items)

    def detalle(self):
        lineas = []
        for item in self.items:
            lineas.append(f"{item.cantidad} x {item.producto.nombre} @ {item.precio_unitario:.2f} = {item.cantidad * item.precio_unitario:.2f}")
        return lineas


class OrderItem(Base):
    __tablename__ = 'order_items'
    id = Column(Integer, primary_key=True)
    pedido_id = Column(Integer, ForeignKey('pedidos.id'), nullable=False)
    producto_id = Column(Integer, ForeignKey('productos.id'), nullable=False)
    cantidad = Column(Integer, nullable=False)
    precio_unitario = Column(Float, nullable=False)

    pedido = relationship('Pedido', back_populates='items')
    producto = relationship('Producto', back_populates='items')

In [3]:
# Crear la base en memoria o en archivo
engine = create_engine('sqlite:///:memory:', echo=True)

In [4]:
# Crear todas las tablas
Base.metadata.create_all(engine)

2025-05-07 17:04:57,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-07 17:04:57,958 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("usuarios")
2025-05-07 17:04:57,961 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-07 17:04:57,965 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("usuarios")
2025-05-07 17:04:57,967 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-07 17:04:57,972 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("productos")
2025-05-07 17:04:57,976 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-07 17:04:57,980 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("productos")
2025-05-07 17:04:57,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-07 17:04:57,987 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("pedidos")
2025-05-07 17:04:57,991 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-07 17:04:57,998 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("pedidos")
2025-05-07 17:04:58,001 INFO sqlalchemy.engine.Engine [raw s

In [5]:
# Crear una sesión
Session = sessionmaker(bind=engine)
session = Session()

Datos Ficticios para Cargar

In [6]:
# Crear usuarios y productos
u1 = Usuario(nombre='Ana', email='ana@example.com', direccion='Calle Falsa 123')
u2 = Usuario(nombre='Luis', email='luis@example.com', direccion='Av. Siempre Viva 742')

p1 = Producto(nombre='Teclado', descripcion='Teclado mecánico', precio=45.50, stock=10)
p2 = Producto(nombre='Mouse', descripcion='Mouse óptico', precio=20.00, stock=20)
p3 = Producto(nombre='Monitor', descripcion='24 pulgadas LED', precio=120.99, stock=5)

session.add_all([u1, u2, p1, p2, p3])
# session.commit()

# Crear el pedido y sus items *antes* del commit
pedido = Pedido(usuario=u1, estado='pendiente')

item1 = OrderItem(producto=p1, cantidad=1, precio_unitario=p1.precio)
item2 = OrderItem(producto=p2, cantidad=2, precio_unitario=p2.precio)

pedido.items.extend([item1, item2])

session.add(pedido)
session.commit()

2025-05-07 17:05:08,380 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-07 17:05:08,397 INFO sqlalchemy.engine.Engine INSERT INTO productos (nombre, descripcion, precio, stock) VALUES (?, ?, ?, ?) RETURNING id
2025-05-07 17:05:08,404 INFO sqlalchemy.engine.Engine [generated in 0.00102s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('Teclado', 'Teclado mecánico', 45.5, 10)
2025-05-07 17:05:08,415 INFO sqlalchemy.engine.Engine INSERT INTO productos (nombre, descripcion, precio, stock) VALUES (?, ?, ?, ?) RETURNING id
2025-05-07 17:05:08,419 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('Mouse', 'Mouse óptico', 20.0, 20)
2025-05-07 17:05:08,428 INFO sqlalchemy.engine.Engine INSERT INTO productos (nombre, descripcion, precio, stock) VALUES (?, ?, ?, ?) RETURNING id
2025-05-07 17:05:08,436 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('Monitor', '24 pulgadas LED', 120.99, 5)
2025-05-07 17:05:08,456

In [8]:
pedido.total()

2025-05-07 17:10:14,653 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-07 17:10:14,668 INFO sqlalchemy.engine.Engine SELECT pedidos.id AS pedidos_id, pedidos.usuario_id AS pedidos_usuario_id, pedidos.fecha AS pedidos_fecha, pedidos.estado AS pedidos_estado 
FROM pedidos 
WHERE pedidos.id = ?
2025-05-07 17:10:14,671 INFO sqlalchemy.engine.Engine [generated in 0.00373s] (1,)
2025-05-07 17:10:14,686 INFO sqlalchemy.engine.Engine SELECT order_items.id AS order_items_id, order_items.pedido_id AS order_items_pedido_id, order_items.producto_id AS order_items_producto_id, order_items.cantidad AS order_items_cantidad, order_items.precio_unitario AS order_items_precio_unitario 
FROM order_items 
WHERE ? = order_items.pedido_id
2025-05-07 17:10:14,690 INFO sqlalchemy.engine.Engine [generated in 0.00377s] (1,)


85.5

In [9]:
pedido.detalle()

2025-05-07 17:10:51,401 INFO sqlalchemy.engine.Engine SELECT productos.id AS productos_id, productos.nombre AS productos_nombre, productos.descripcion AS productos_descripcion, productos.precio AS productos_precio, productos.stock AS productos_stock 
FROM productos 
WHERE productos.id = ?
2025-05-07 17:10:51,407 INFO sqlalchemy.engine.Engine [generated in 0.00560s] (1,)
2025-05-07 17:10:51,418 INFO sqlalchemy.engine.Engine SELECT productos.id AS productos_id, productos.nombre AS productos_nombre, productos.descripcion AS productos_descripcion, productos.precio AS productos_precio, productos.stock AS productos_stock 
FROM productos 
WHERE productos.id = ?
2025-05-07 17:10:51,429 INFO sqlalchemy.engine.Engine [cached since 0.02815s ago] (2,)


['1 x Teclado @ 45.50 = 45.50', '2 x Mouse @ 20.00 = 40.00']

Pedido para luis que quiere 2 monitores un teclado y un mouse

In [10]:
# 1. Obtener al usuario "Luis"
luis = session.query(Usuario).filter_by(nombre='Luis').first()

# 2. Obtener los productos necesarios
teclado = session.query(Producto).filter_by(nombre='Teclado').first()
mouse = session.query(Producto).filter_by(nombre='Mouse').first()
monitor = session.query(Producto).filter_by(nombre='Monitor').first()

# 3. Crear el nuevo pedido
pedido_luis = Pedido(usuario=luis, estado='pendiente')

# 4. Crear los ítems del pedido
item1 = OrderItem(producto=teclado, cantidad=1, precio_unitario=teclado.precio)
item2 = OrderItem(producto=mouse, cantidad=1, precio_unitario=mouse.precio)
item3 = OrderItem(producto=monitor, cantidad=2, precio_unitario=monitor.precio)

# Asignar los ítems al pedido
pedido_luis.items.extend([item1, item2, item3])

# 5. Agregar el pedido a la sesión y guardar todo
session.add(pedido_luis)
session.commit()

print("Pedido creado correctamente para Luis.")

2025-05-07 17:15:50,511 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email, usuarios.direccion AS usuarios_direccion, usuarios.fecha_creacion AS usuarios_fecha_creacion 
FROM usuarios 
WHERE usuarios.nombre = ?
 LIMIT ? OFFSET ?
2025-05-07 17:15:50,517 INFO sqlalchemy.engine.Engine [generated in 0.00705s] ('Luis', 1, 0)
2025-05-07 17:15:50,530 INFO sqlalchemy.engine.Engine SELECT productos.id AS productos_id, productos.nombre AS productos_nombre, productos.descripcion AS productos_descripcion, productos.precio AS productos_precio, productos.stock AS productos_stock 
FROM productos 
WHERE productos.nombre = ?
 LIMIT ? OFFSET ?
2025-05-07 17:15:50,542 INFO sqlalchemy.engine.Engine [generated in 0.01240s] ('Teclado', 1, 0)
2025-05-07 17:15:50,558 INFO sqlalchemy.engine.Engine SELECT productos.id AS productos_id, productos.nombre AS productos_nombre, productos.descripcion AS productos_descripcion, productos.

In [12]:
for item in pedido_luis.detalle():
    print (item)
print(pedido_luis.total())

1 x Teclado @ 45.50 = 45.50
1 x Mouse @ 20.00 = 20.00
2 x Monitor @ 120.99 = 241.98
307.48


In [15]:
productos = session.query(Producto)

In [20]:
for producto in productos.all():
    print(producto.nombre, producto.stock)

2025-05-07 17:24:19,962 INFO sqlalchemy.engine.Engine SELECT productos.id AS productos_id, productos.nombre AS productos_nombre, productos.descripcion AS productos_descripcion, productos.precio AS productos_precio, productos.stock AS productos_stock 
FROM productos
2025-05-07 17:24:19,966 INFO sqlalchemy.engine.Engine [cached since 137.6s ago] ()
Teclado 10
Mouse 20
Monitor 5


📄 Leer datos (Read)

In [22]:
# Obtener todos los pedidos de Ana
ana = session.query(Usuario).filter_by(nombre='Ana').first()
for p in ana.pedidos:
    print("Pedido ID:", p.id)
    print("Estado : ", p.estado)
    print("Total:", p.total())
    print("Detalle:")
    for linea in p.detalle():
        print("  ", linea)

2025-05-07 17:32:03,526 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email, usuarios.direccion AS usuarios_direccion, usuarios.fecha_creacion AS usuarios_fecha_creacion 
FROM usuarios 
WHERE usuarios.nombre = ?
 LIMIT ? OFFSET ?
2025-05-07 17:32:03,532 INFO sqlalchemy.engine.Engine [cached since 973s ago] ('Ana', 1, 0)
Pedido ID: 1
Estado :  pendiente
Total: 85.5
Detalle:
   1 x Teclado @ 45.50 = 45.50
   2 x Mouse @ 20.00 = 40.00


📝 Modificar datos (Update)

In [24]:
p.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x234db41c4c0>,
 'id': 1,
 'estado': 'pendiente',
 'usuario_id': 1,
 'fecha': datetime.datetime(2025, 5, 7, 20, 5, 8, 522650),
 'items': [<__main__.OrderItem at 0x234ddad0e80>,
  <__main__.OrderItem at 0x234ddad1390>]}

In [25]:
# Cambiar estado del pedido
p.estado = 'enviado'
session.commit()

2025-05-07 17:35:46,175 INFO sqlalchemy.engine.Engine UPDATE pedidos SET estado=? WHERE pedidos.id = ?
2025-05-07 17:35:46,181 INFO sqlalchemy.engine.Engine [generated in 0.00575s] ('enviado', 1)
2025-05-07 17:35:46,189 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
ana = session.query(Usuario).filter_by(nombre='Ana').first()
for p in ana.pedidos:
    print("Pedido ID:", p.id)
    print("Estado : ", p.estado)
    print("Total:", p.total())
    print("Detalle:")
    for linea in p.detalle():
        print("  ", linea)

2025-05-07 17:36:38,571 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email, usuarios.direccion AS usuarios_direccion, usuarios.fecha_creacion AS usuarios_fecha_creacion 
FROM usuarios 
WHERE usuarios.nombre = ?
 LIMIT ? OFFSET ?
2025-05-07 17:36:38,575 INFO sqlalchemy.engine.Engine [cached since 1248s ago] ('Ana', 1, 0)
Pedido ID: 1
Estado :  enviado
Total: 85.5
Detalle:
   1 x Teclado @ 45.50 = 45.50
   2 x Mouse @ 20.00 = 40.00


❌ Eliminar datos (Delete)

In [28]:
# Eliminar un producto
producto_a_eliminar = session.query(Producto).filter_by(nombre='Monitor').first()
session.delete(producto_a_eliminar)
session.commit()

2025-05-07 17:38:13,523 INFO sqlalchemy.engine.Engine SELECT productos.id AS productos_id, productos.nombre AS productos_nombre, productos.descripcion AS productos_descripcion, productos.precio AS productos_precio, productos.stock AS productos_stock 
FROM productos 
WHERE productos.nombre = ?
 LIMIT ? OFFSET ?
2025-05-07 17:38:13,527 INFO sqlalchemy.engine.Engine [cached since 1343s ago] ('Monitor', 1, 0)
2025-05-07 17:38:13,548 INFO sqlalchemy.engine.Engine SELECT order_items.id AS order_items_id, order_items.pedido_id AS order_items_pedido_id, order_items.producto_id AS order_items_producto_id, order_items.cantidad AS order_items_cantidad, order_items.precio_unitario AS order_items_precio_unitario 
FROM order_items 
WHERE ? = order_items.producto_id
2025-05-07 17:38:13,558 INFO sqlalchemy.engine.Engine [generated in 0.01095s] (3,)
2025-05-07 17:38:13,575 INFO sqlalchemy.engine.Engine UPDATE order_items SET producto_id=? WHERE order_items.id = ?
2025-05-07 17:38:13,580 INFO sqlalchemy

IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: order_items.producto_id
[SQL: UPDATE order_items SET producto_id=? WHERE order_items.id = ?]
[parameters: (None, 5)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

creo que se debe que tengo un pedido donde esta ese producto, puede ser '


ChatGPT dijo:
Exacto, el error que estás viendo se debe a que estás intentando eliminar un producto que está referenciado en otra tabla, específicamente en order_items (ítems de pedido), y esa relación tiene una restricción NOT NULL en el campo producto_id.

Qué está pasando
Cuando haces:

```python
session.delete(producto_a_eliminar)
session.commit()
```

SQLAlchemy intenta actualizar los registros de order_items que hacen referencia a ese producto, probablemente porque tienes una relación definida con cascade="all" o similar, y no se puede establecer producto_id = NULL porque el campo es NOT NULL.

Cómo solucionarlo
Tienes varias opciones, dependiendo del comportamiento que desees:

 ✅ Opción 1: Prevenir la eliminación si el producto está en uso

```python
producto = session.query(Producto).filter_by(nombre='Monitor').first()

# Verificar si hay pedidos con ese producto
if producto and producto.order_items:
    print("No se puede eliminar el producto, está en pedidos existentes.")
else:
    session.delete(producto)
    session.commit()
```

- ✅ Opción 2: Eliminar también los ítems relacionados (si estás seguro)
Esto borra los registros de order_items que lo usan:

```python
producto = session.query(Producto).filter_by(nombre='Monitor').first()

if producto:
    for item in producto.order_items:
        session.delete(item)
    session.delete(producto)
    session.commit()
```

- ✅ Opción 3: Definir ON DELETE CASCADE en la base de datos

Si usás SQLite, necesitas declarar las claves foráneas con ondelete="CASCADE" y activar las foreign keys así:

```python
from sqlalchemy import event
from sqlalchemy.engine import Engine

@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()
```

Y en tu modelo de OrderItem:

```python
pedido_id = Column(Integer, ForeignKey('pedidos.id', ondelete='CASCADE'))
producto_id = Column(Integer, ForeignKey('productos.id', ondelete='CASCADE'))
```

Pero ten en cuenta que para que esto funcione bien, debes haber creado la base con esa configuración desde el principio.

¿Querés que te ayude a revisar tus modelos para ver si podemos aplicar esta solución de forma segura?